In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from spacy.lang.ru.stop_words import STOP_WORDS

df_items = pd.read_csv('./processed_data/items.csv')
df_users = pd.read_csv('./processed_data/users.csv')

## Items

In [2]:
df_items.sample(1)

,item_id,content_type,title,genres,age_rating,keywords,views
14081,1263,film,Пришла и говорю,"документальное, русские, мюзиклы",12,"Пришла, говорю, 1985, СССР",1


In [3]:
def tokenize(line: str):
    return [word for word in line.split(', ') 
            if not word.isnumeric() and word not in {'', 'nan'}]

keywords_vectorizer = TfidfVectorizer(
    tokenizer=tokenize, token_pattern=None,
    max_features=100, stop_words=list(STOP_WORDS))

X_keywords = keywords_vectorizer.fit_transform(
    df_items['keywords'].values.astype('U')).todense()

X_content_type = (df_items['content_type'] == 'film').\
    astype(int).values.reshape(-1, 1)

age_rating_encoder = OneHotEncoder(sparse_output=False)
X_age_rating = age_rating_encoder.fit_transform(
    df_items['age_rating'].values.astype('U').reshape(-1, 1))

genres_vectorizer = TfidfVectorizer(tokenizer=tokenize, token_pattern=None)
X_genres = genres_vectorizer.fit_transform(
    df_items['genres'].values.astype('U')).todense()

In [4]:
keywords_vectorizer.get_feature_names_out()

array(['бандиты', 'бар', 'беременность', 'близких', 'бой', 'больница',
       'борьба', 'брак', 'великобритания', 'взрыв', 'война', 'выбор',
       'выживание', 'выстрел в грудь', 'гангстеры', 'германия', 'героизм',
       'героя', 'дела', 'деньги', 'детей', 'дети', 'дочери', 'дружба',
       'друзья', 'женщины', 'жены', 'жертвы', 'жестокость', 'животные',
       'жизненных', 'жизнь', 'загадки', 'задание', 'замедленная сцена',
       'злом', 'интриги', 'италия', 'канада', 'конфликт', 'конфликты',
       'кровь', 'любовные', 'любовь', 'матери', 'месть', 'миссия', 'мужа',
       'мужчина с обнаженной грудью', 'мужчины', 'название', 'обман',
       'отношения', 'отношениях', 'отца', 'отцы', 'паре', 'пороки',
       'потеря', 'правопорядка', 'преображения', 'преодоление',
       'преследования', 'преступления', 'приоритетов', 'проблемы',
       'прошлое', 'путешествия', 'работа', 'расследование', 'родителей',
       'романтические', 'россия', 'сверхъестественное', 'семейные',
       'семей

In [5]:
pd.Series(
    data=np.squeeze(np.asarray(X_keywords.sum(axis=0))),
    index=keywords_vectorizer.get_feature_names_out()
    ).sort_values(ascending=False)[:10]

россия                     2464.969923
соединенные штаты          1540.678985
отношения                  1213.487046
франция                    1138.964494
сша                        1018.792042
ссср                        669.475555
любовь                      573.267843
дружба                      560.706653
соединенное королевство     477.921714
женщины                     440.855611
dtype: float64

In [6]:
X_movies = np.hstack([X_content_type, X_genres, X_age_rating, X_keywords])
X_movies.shape

(15008, 202)

## Users

In [7]:
df_users.sample(1)

,user_id,age,income,sex,kids_flg
308804,247148,age_25_34,income_20_40,М,0


In [8]:
age_encoder = OneHotEncoder(sparse_output=False)
X_age = age_encoder.fit_transform(
    df_users['age'].values.astype('U').reshape(-1, 1))

income_encoder = OneHotEncoder(sparse_output=False)
X_income = income_encoder.fit_transform(
    df_users['income'].values.astype('U').reshape(-1, 1))

X_sex = (df_users['sex'] == 'М').astype(int).values.reshape(-1, 1)

In [9]:
X_users = np.hstack([X_age, X_income, X_sex,
    df_users['kids_flg'].to_numpy().reshape(-1, 1)])
X_users.shape

(744288, 14)

In [10]:
np.savez_compressed('encoded_data',
                    X_users=X_users.astype(np.float32),
                    X_movies=X_movies.astype(np.float32))